# Ejemplo de Inferencia

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install ultralytics

In [ ]:
import os
import shutil
import json
from ultralytics import YOLO

## Ajustes Iniciales

In [ ]:
# Nombre del alumno
student_name = "lucas_bruge"

# Ruta al archivo de pesos
model_path = "/content/drive/MyDrive/TUIA/CV/2024-01_practico/model/weights/best.pt"

# Ruta al directorio que contiene las imagenes
imgs_dir = "/content/drive/MyDrive/TUIA/CV/2024-01_practico/data/eval/images/val"

# Ruta al directorio de destino de las detecciones
base_dir = "/content/drive/MyDrive/TUIA/CV/2024-01_practico/data/out"
dets_dir = os.path.join(base_dir, student_name)

In [ ]:
# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)

os.makedirs(dets_dir)

## Inferencia

### Formato YOLOv5 para las salidas de las inferencias

Estas son las celdas para que el estudiante complete con su código de inferencia.

Debe generar un archivo con las detecciones en formato YOLOv5, donde cada línea contiene:
 cls, xc, yx, w, h, c

* **cls**: número de índice de la clase

* **xc**: coordenada x al centro de la caja delimitadora (bbox)

* **yx**: coordenada y al centro de la caja delimitadora (bbox)

* **w**: ancho de la caja delimitadora

* **h**: alto de la caja delimitadora

* **c**: certeza del modelo sobre la clasificacíon (confidence)


 Todas las coordenadas deben ser relativas: [0 ... 1]

NOTA: Si utiliza un modelo YoloV8 de ultralytics, pueder llamar al método result.save_txt(file_name, save_conf=True)

### Formato para los archivos de envido.json

Además de las detecciones anteriores, debe existir un archivo llamado envido.json con la siguiente estructura:

```json
{
    "IMG_20240630_174514649_HDR.jpg": {
        "total_cards": 3,
        "cards": {
            "E": [],
            "C": [
                12
            ],
            "B": [
                11
            ],
            "O": [
                10
            ]
        },
        "points": 0,
        "figure": "N/A"
    },
    "IMG_20240630_173918579.jpg": {
        "total_cards": 2,
        "cards": {
            "E": [],
            "C": [],
            "B": [],
            "O": [
                9,
                8
            ]
        },
        "points": 0,
        "figure": "N/A"
    },
  ...
}
```

Esto indica que hay un total de tres cartas, que son el 5 y el 1 de Copas, y el 2 de Basto, que dan 26 puntos de envido de Copa.

Este archivo debe contener una clave con el nombre de archivo, donde cada elemento es a su vez un objeto cons las claves detalladas.

Revise el archivo gt_envido.json para analizar todas las posibilidades.

### DEMO: cálculo del envido

A continuación hay una implementación del cálculo de los puntos a modo de ejemplo, que sirve de referencia para evaluar el algoritmo de evaluación.

**ESTA ES LA SECCIÓN QUE CADA ALUMNO REEMPLAZARIA CON SU CODIGO**

## Escritura del archivo envido.json

In [ ]:
with open(os.path.join(dets_dir, "envido.json"), "w") as jf:
    json.dump(card_ds_file, jf, indent=4)